## GloVe

In [2]:
import re
import urllib.request
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f1ecd82e890>)

In [3]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

parse_text = '\n'.join(target_text.xpath('//content/text()')) # <content> ~ </content> 부분의 내용을 개행하며 하나의 문자열로 통합
content_text = re.sub(r'\([^)]*\)', '', parse_text)           # 괄호로 묶인 내부에 있는 불필요한 부분 제거
sent_text = sent_tokenize(content_text)                       # 문자열을 sentence tokenizer로 문장 토큰화

normalized_text = []
for string in sent_text:
  tokens = re.sub(r"[^a-z0-9]+", " ", string.lower()) # 구두점 제거, 소문자 변환
  normalized_text.append(tokens)                      # 정규화된 문장 토큰들로 만들고

result = [word_tokenize(sentence) for sentence in normalized_text]  # 각 문장마다 단어 토큰화를 진행하여 총 단어 토큰들로 이루어진 Corpus 생성
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
# 2024년 7월 colab 환경 기준, glove_python, glove_python_binary 모두 안 된다
!pip install glove-python3

In [7]:
from glove import Corpus, Glove

corpus = Corpus()

corpus.fit(result, window=5) # 단어 토큰화된 result를 기반으로하고 window를 5로 잡아 corpus를 생성
glove = Glove(no_components=100, learning_rate=0.05)
# no_components : 워드 임베딩 벡터의 차원

glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [18]:
corpus.matrix.shape # co-occurence matrix의 차원
                    # 즉, 54775개의 단어가 corpus를 구성하고 있다고 볼 수 있다

(54775, 54775)

In [8]:
print(glove.most_similar("man"))

[('woman', 0.962044604125184), ('guy', 0.9017877179675742), ('girl', 0.8511967695295493), ('young', 0.8431801139894113)]


In [9]:
print(glove.most_similar("boy"))

[('girl', 0.936422254880417), ('woman', 0.8388378818393863), ('kid', 0.8290882481640035), ('man', 0.8250209209649827)]


In [10]:
print(glove.most_similar("university"))

[('harvard', 0.8809321437038237), ('mit', 0.843707551558188), ('cambridge', 0.8404076394004669), ('stanford', 0.8366046714180211)]


In [11]:
print(glove.most_similar("water"))

[('air', 0.8416346493060578), ('clean', 0.8337566160174978), ('fresh', 0.8322980136847533), ('electricity', 0.8105004538118061)]


In [12]:
print(glove.most_similar("physics"))

[('chemistry', 0.9032726357073866), ('beauty', 0.8753921742667087), ('economics', 0.8690628828914557), ('biology', 0.8632811664952816)]


In [13]:
print(glove.most_similar("muslce")) # 잘못 입력 했을 때

Exception: Word not in dictionary

In [14]:
print(glove.most_similar("muscle"))

[('tissue', 0.8602045938691039), ('nerve', 0.8208525721280927), ('foreign', 0.7717823414490498), ('bone', 0.7674321580498014)]


In [15]:
print(glove.most_similar("clean"))

[('fresh', 0.8351820892499696), ('water', 0.833756616017498), ('heat', 0.8247906158059961), ('air', 0.8048206866349596)]


## FastText vs Word2Vec

In [3]:
len(result) # 기존에 전처리해둔 데이터셋

273424

In [28]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0) # 450만개 가량의 토큰에 대한 Word2Vec 학습(40초 정도 소요)

In [29]:
# Word2Vec의 경우 오타나 이상한 어휘에 대해 OOV로 인한 문제가 발생
model.wv.most_similar("electrofishing")

KeyError: "Key 'electrofishing' not present in vocabulary"

In [31]:
from gensim.models import FastText

fast_text_model = FastText(result, vector_size=100, window=5, min_count=5, workers=4, sg=0) # 동일한 옵션에서 FastText 모델로 학습

In [32]:
# Word2Vec과 다르게 Subwords에 대해 학습하였기 때문에 OOV, Rare Word에 robust하다
fast_text_model.wv.most_similar("electrofishing")

[('licensing', 0.913245677947998),
 ('fishing', 0.9109775424003601),
 ('operating', 0.9012295007705688),
 ('recycling', 0.9002954363822937),
 ('sprinting', 0.9002386331558228),
 ('skateboarding', 0.8984874486923218),
 ('ceiling', 0.8978874087333679),
 ('flourishing', 0.8963640928268433),
 ('transplanting', 0.8961915373802185),
 ('flushing', 0.89476478099823)]

"apple"에 대해서 Word2Vec은 Apple 회사 관련한 키워드가 많이 보이며, FastText의 경우 subwords를 고려하여 apple과 유사한 표현들, 예를 들어 pineapple 등의 유사도가 높게 나왔다

In [33]:
model.wv.most_similar("apple")

[('alarm', 0.7501398324966431),
 ('icon', 0.7498602271080017),
 ('ipad', 0.7389275431632996),
 ('iphone', 0.7355656027793884),
 ('f', 0.7285366654396057),
 ('ipod', 0.727858304977417),
 ('elephant', 0.7243888974189758),
 ('mp3', 0.7219675779342651),
 ('airplane', 0.7171599268913269),
 ('app', 0.7150890231132507)]

In [34]:
fast_text_model.wv.most_similar("apple")

[('pineapple', 0.9224739074707031),
 ('grapple', 0.9130294919013977),
 ('nipple', 0.8849395513534546),
 ('cripple', 0.880841076374054),
 ('temple', 0.870653510093689),
 ('app', 0.8558655977249146),
 ('purple', 0.8550805449485779),
 ('ripple', 0.8473930954933167),
 ('ample', 0.8458758592605591),
 ('triple', 0.8452322483062744)]

Word2Vec에는 Typo가 있는 'appple'에 대해 OOV 문제가 발생하는 반면,   
FastText는 기존의 apple과 동일한 유사도를 보이는 것을 확인할 수 있다

In [35]:
model.wv.most_similar("appple")

KeyError: "Key 'appple' not present in vocabulary"

In [36]:
fast_text_model.wv.most_similar("apple")

[('pineapple', 0.9224739074707031),
 ('grapple', 0.9130294919013977),
 ('nipple', 0.8849395513534546),
 ('cripple', 0.880841076374054),
 ('temple', 0.870653510093689),
 ('app', 0.8558655977249146),
 ('purple', 0.8550805449485779),
 ('ripple', 0.8473930954933167),
 ('ample', 0.8458758592605591),
 ('triple', 0.8452322483062744)]

## 자모 단위 한국어 FastText 학습

In [ ]:
# 이것저것 다 시도해봤는데 Colab에서는 이렇게 하면 Mecab 오류가 발생하지 않음
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
!pip install hgtk # 한글 자모 단위 처리 패키지 hgtk 설치

In [ ]:
# fasttext 설 치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

In [6]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk
from konlpy.tag import Mecab

# 네이버 쇼핑 리뷰 데이터 로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x790c4bd463b0>)

In [7]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data))

전체 리뷰 개수 : 200000


In [8]:
total_data.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [9]:
def word_to_jamo(token):
  def to_special_token(jamo):
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)
      # 자모가 빈 문자일 경우 특수 문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [10]:
word_to_jamo('남동생')

'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

In [11]:
word_to_jamo('여동생') # '여'에 종성이 없어서 -가 들어간 것을 확인 가능

'ㅇㅕ-ㄷㅗㅇㅅㅐㅇ'

In [12]:
test_sentence = "선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다."

mecab = Mecab() # 형태소 분석기
print("단순 형태소 분석 결과 :", mecab.morphs(test_sentence))

def tokenize_by_jamo(s):
  return [word_to_jamo(token) for token in mecab.morphs(s)]

print("형태소 분석 + 자모 분해 :", tokenize_by_jamo(test_sentence))

단순 형태소 분석 결과 : ['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']
형태소 분석 + 자모 분해 : ['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [13]:
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_list()):
  tokenized_sample = tokenize_by_jamo(sample) # 자모 단위로 분해
  tokenized_data.append(tokenized_sample)

print(tokenized_data[0])

100%|██████████| 200000/200000 [01:02<00:00, 3190.28it/s]

['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']


In [14]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)가 아닐 때
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면, 초중종성을 하나의 토큰으로 간주
    else:
      tokenized_jamo.append(jamo_sequence[index:index+3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:
      # 초, 중, 종성이 다 있는 경우
      if len(jamo) == 3:
        if jamo[2] == '-':
          # 종성이 없는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 있는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'
  return word

In [15]:
jamo_to_word('ㄴㅏㅁㄷㅗㅇㅅㅐㅇ')

'남동생'

In [16]:
import fasttext

with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:01<00:00, 197692.72 line/s]


In [17]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')
model.save_model("fasttext.bin")            # 모델 저장
model = fasttext.load_model("fasttext.bin") # 모델 로드

In [18]:
model[word_to_jamo('남동생')] # 실제로는 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'이라는 입력

array([-0.06107997,  0.6434101 ,  0.20715   ,  0.29333124,  0.7893051 ,
       -0.40490672,  0.6929343 , -0.44864348, -0.24650103, -0.20013273,
        0.30655596,  0.18253864, -0.7267484 , -0.00349172,  0.4574451 ,
       -0.23841374,  0.29909563,  0.19065087, -0.6804761 , -0.5404831 ,
       -0.06967576, -0.8608843 ,  0.8253546 , -0.22332686, -1.0310053 ,
       -0.6039053 ,  0.8116493 ,  0.20559157,  0.3903545 ,  1.0808167 ,
        0.41745734, -0.06997378, -0.05163563,  0.41713077,  0.527806  ,
       -0.30835825,  0.31510252, -0.50626683, -0.01788406, -0.19891815,
        0.83161736,  0.6717083 , -0.48440403,  1.0042284 , -0.11523026,
        0.11539069,  0.81462663, -0.09036586, -0.73975825, -0.2623649 ,
        0.12626076,  0.406552  , -0.10470987,  0.3460048 , -0.5163345 ,
       -0.24508008,  0.18592907, -0.39331028,  0.25789505,  0.0759757 ,
        0.35949537,  0.05539207,  0.9095674 ,  0.5652896 , -0.3367405 ,
       -0.63889617, -0.74739826, -0.3096703 , -0.45060015, -0.79

In [20]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8896892666816711, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8556913733482361, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7981497049331665, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7781562805175781, 'ㅊㅣㄴㄱㅜ-'),
 (0.7702001929283142, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7277362942695618, 'ㅈㅗ-ㅋㅏ-'),
 (0.7170624136924744, 'ㄴㅏㅁㅇㅏ-'),
 (0.712027907371521, 'ㅎㅏㄱㅅㅐㅇ'),
 (0.7095516324043274, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.6981350183486938, 'ㅇㅓㄴㄴㅣ-')]

In [21]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [22]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))

[('동생', 0.8896892666816711), ('남친', 0.8556913733482361), ('남편', 0.7981497049331665), ('친구', 0.7781562805175781), ('생일', 0.7702001929283142), ('조카', 0.7277362942695618), ('남아', 0.7170624136924744), ('학생', 0.712027907371521), ('중학생', 0.7095516324043274), ('언니', 0.6981350183486938)]


In [23]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동쉥'), k=10)))

[('남동생', 0.9047474265098572), ('남친', 0.8278025984764099), ('남매', 0.7707632780075073), ('남짓', 0.7279080748558044), ('남녀', 0.7082976698875427), ('남아', 0.7055363655090332), ('남김', 0.6959303617477417), ('남긴', 0.6883314251899719), ('남여', 0.6816936135292053), ('남편', 0.680534303188324)]


In [24]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동셍ㅋ'), k=10)))

[('남동생', 0.8317911028862), ('남친', 0.7488435506820679), ('남아', 0.6314992308616638), ('남짓', 0.629278838634491), ('동생', 0.6279979944229126), ('남녀', 0.61879563331604), ('남매', 0.6171734929084778), ('남김', 0.6137759685516357), ('남여', 0.6029059886932373), ('생일', 0.601040780544281)]


In [25]:
print(transform(model.get_nearest_neighbors(word_to_jamo('난동생'), k=10)))

[('남동생', 0.8642020225524902), ('난생', 0.832841694355011), ('남편', 0.8116939067840576), ('남친', 0.7826412320137024), ('동생', 0.7780384421348572), ('남아', 0.7360055446624756), ('중학생', 0.7287946343421936), ('남매', 0.7133886218070984), ('학생', 0.7039417028427124), ('남자', 0.6721429228782654)]


In [26]:
print(transform(model.get_nearest_neighbors(word_to_jamo('낫동생'), k=10)))

[('남동생', 0.929941713809967), ('동생', 0.8905754685401917), ('남편', 0.7983264923095703), ('남친', 0.7660068869590759), ('친구', 0.7500053644180298), ('중학생', 0.7456657886505127), ('학생', 0.7277404069900513), ('난생', 0.7130797505378723), ('조카', 0.7127023339271545), ('생일', 0.7112838625907898)]


In [27]:
print(transform(model.get_nearest_neighbors(word_to_jamo('납동생'), k=10)))

[('남동생', 0.9161667823791504), ('동생', 0.8544796109199524), ('남편', 0.8233013153076172), ('남친', 0.8008506894111633), ('난생', 0.7529798746109009), ('중학생', 0.7363580465316772), ('친구', 0.7217780947685242), ('학생', 0.7159159779548645), ('남아', 0.715417206287384), ('조카', 0.707827627658844)]


In [28]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품질'), k=10)))

[('품질', 0.8579558730125427), ('음질', 0.8266991376876831), ('땜질', 0.7590718269348145), ('찜질', 0.7277037501335144), ('퀄리티', 0.7079712152481079), ('사포질', 0.7016077637672424), ('군것질', 0.6904250383377075), ('성질', 0.6781318187713623), ('다림질', 0.659640371799469), ('퀄러티', 0.6441708207130432)]


In [29]:
print(transform(model.get_nearest_neighbors(word_to_jamo('고품쥘'), k=10)))

[('고품질', 0.8423395156860352), ('재고품', 0.7486403584480286), ('고퀄', 0.7318102717399597), ('반제품', 0.7153773307800293), ('중고품', 0.7060855031013489), ('소모품', 0.6940242052078247), ('소지품', 0.6910591125488281), ('재품', 0.6895123720169067), ('화학제품', 0.6871013641357422), ('곪', 0.6835593581199646)]
